# Preliminaries

Importing python packages and setting display parameters

In [1]:
import math as mt
import random as rnd
import numpy as np

from deap import base, creator, tools

import numba
from numba import jit
import joblib

import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import pandas as pd
import statistics as stats

In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = "retina"

plt.style.use("default")
plt.style.use("bmh")
# plt.rcParams.update({"figure.autolayout": True})
plt.rcParams["figure.figsize"] = (12, 9)
mpl.rcParams["figure.dpi"] = 100
mpl.rcParams["savefig.dpi"] = 100

In [3]:
pd.set_option("display.latex.repr", True)

In [4]:
pd.set_option("display.latex.longtable", True)

# Fitness Landscape Definition

In [5]:
# Problem domain
x_min = -15
x_max = 15
y_min = -15
y_max = 15

# Known minimum
x_point = -1
y_point = -1


domain = (x_min, x_max, y_min, y_max)
point = (x_point, y_point)
img_size = (8.5, 4.25)

# Problem definition


@jit
def g_fun(x, y):
    mag = np.sqrt(x ** 2.0 + y ** 2.0)
    val = -(50.0 * np.sinc(mag / np.pi) - mag)
    return val.item()


@jit
def f_fun(x, y):
    D = 2
    alpha = 1 / 8

    x = (x - 5) / 6
    y = (y - 5) / 6

    a = np.abs(x ** 2 + y ** 2 - D) ** (alpha * D)
    b = (0.5 * (x ** 2 + y ** 2) + (x + y)) / D

    return a + b + 0.5

In [6]:
@jit
def evaluate(individual):
    x = individual[0]
    y = individual[1]
    fitness = f_fun(x, y)
    return (fitness,)

In [7]:
# Testing the minimum
print(f_fun(-1, -1))

0.0


In [8]:
# Testing the function
print(f_fun(-1.0, -1.0), f_fun(-11.0, -9.0), f_fun(11.0, 3.0), f_fun(-6.0, 9.0))

0.0 2.941368139329697 2.082094654525758 2.0272404522465126


# Running the Evolutionary Algorithm

## Setting the EA's parameters

From TLFL

In [9]:
# Algorithm parameters
# Number of replicates, and generations per experiment
rep_end = 40
births_end = 120e3

# Genes
gen_size = 2
# Population size
pop_size = 160
# Progeny and parents size
b_ratio = 2
par_size = b_ratio * pop_size

# Progeny parameters
## Crossover probability per gene
cx_pb = 0.3
## Mutation probability per gene
mut_pb = 0.5
## Mutation strength
mut_sig = 0.5

# Selection by tournament
# Tournament size parent selection
k_par = 4
# Tournament size survivor selection
k_sur = 6

## Defining the EA elements

We define that the fitness is related to a minimizing problem, and that each individual is represented with a list of numbers

In [10]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

We start the DEAP toolset. At creation, each individual will have 2 genes of type "float" that are randomly initialized in the range [-15; 15].

In [11]:
toolbox = base.Toolbox()

In [12]:
toolbox.register("attr_float", rnd.uniform, -15, 15)
toolbox.register(
    "individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=gen_size
)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

We add our landscape to the toolset, indicate that the mating will use a uniform crossover on a per-gene basis, that the mutation will be also on a per-gene basis with a value taken from a gaussian distribution, and that parent and survivor selections will use tournament selection.

In [13]:
toolbox.register("evaluate", evaluate)

toolbox.register("mate", tools.cxUniform)
toolbox.register("mutate", tools.mutGaussian, mu=0)

toolbox.register("par_select", tools.selTournament)
toolbox.register("sur_select", tools.selTournament)

We define that for each generation we'll summarize the fitnesses with median, mean, standard deviation, and store the best and worst fitnesses in the generation.

In [14]:
stat = tools.Statistics(key=lambda ind: ind.fitness.values[0])

stat.register("med", stats.median)
stat.register("avg", stats.mean)
stat.register("std", stats.stdev)
stat.register("best", min)
stat.register("worst", max)

We invoque the data recording logbook.

In [15]:
logbook = tools.Logbook()

## Single Run of the EA Experiments
1 Experiment
L-> 1 Parameter set for the experiment.
>L-> 1 Replicate.
>>L-> The replicate is affected due to the randomness seed.

## 100 Executions of the EA
1 Experiment
>L-> 1 Parameter set for the experiment.
>>L-> 100 Replicate.
>>>L-> Each replicate is different due to randomness effects.

### Changing parameters

In [16]:
# Restarting seed
rnd.seed(42)

### Execution

In [17]:
%%time
logbook.clear()

if __name__ == "__main__":
    for rep_n in range(rep_end):
        rep_seed = rnd.randint(0, 999)
        rnd.seed(rep_seed)
        # We initialize the population and evaluate the individuals' fitnesses
        pop = toolbox.population(n=pop_size)
        fitnesses = toolbox.map(toolbox.evaluate, pop)
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit
        # We start the logbook
        record = stat.compile(pop)
        births = len(pop)
        logbook.record(
            rep=rep_n + 1, 
            seed=rep_seed, 
            births=births, 
            **record)

        while births < births_end:
            # Select Parents and clone them as base for offsprings
            parents = toolbox.par_select(pop, k=par_size, tournsize=k_par)
            offspring = [toolbox.clone(ind) for ind in parents]
            births = births + len(offspring)

            # Aplly crossover
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                toolbox.mate(child1, child2, indpb=cx_pb)

            # Apply mutation
            for mutant in offspring:
                toolbox.mutate(mutant, sigma=mut_sig, indpb=mut_pb)
                del mutant.fitness.values

            fitnesses = toolbox.map(toolbox.evaluate, offspring)
            for ind, fit in zip(offspring, fitnesses):
                ind.fitness.values = fit

            pop = toolbox.sur_select((pop + offspring), k=pop_size, tournsize=k_sur)

            record = stat.compile(pop)
            logbook.record(
                rep=rep_n + 1, 
                seed=rep_seed, 
                births=births + 1, 
                **record)

C:\Anaconda3\envs\ea_thesis\lib\site-packages\numba\ir_utils.py:1958: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'individual' of function 'evaluate'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-6-8d5bf46f22a6>", line 2:
@jit
def evaluate(individual):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


Wall time: 7min 10s


### Data Analysis

We transform the records into a Data Frame

In [18]:
fitness_res = pd.DataFrame.from_dict(logbook)
fitness_res = fitness_res[
    ["rep", "seed", "births", "avg", "std", "med", "worst", "best"]
]
fitness_res.head()

,rep,seed,births,avg,std,med,worst,best
0,1,654,160,2.303040,0.837745,2.273725,4.301838,0.380553
1,1,654,481,0.999787,0.267537,1.042822,1.735657,0.380553
2,1,654,801,0.565522,0.132523,0.567964,0.948049,0.279257
3,1,654,1121,0.396316,0.084548,0.380553,0.627210,0.078415
4,1,654,1441,0.314478,0.083426,0.309694,0.638247,0.078415


We filter the values of the last generation

In [19]:
query = fitness_res["births"] >= births_end
fit_120k = fitness_res[query]
query = fitness_res["births"] == 60001
fit_60k = fitness_res[query]
query = fitness_res["births"] == 30241
fit_30k = fitness_res[query]

query_exact = fit_30k["best"] < 1e-6
fit_30k_exact = fit_30k[query_exact]
query_exact = fit_60k["best"] < 1e-6
fit_60k_exact = fit_60k[query_exact]
query_exact = fit_120k["best"] < 1e-6
fit_120k_exact = fit_120k[query_exact]

In [20]:
print(len(fit_30k))
print(len(fit_60k))
print(len(fit_120k))

print(len(fit_30k_exact))
print(len(fit_60k_exact))
print(len(fit_120k_exact))

40
40
40
0
0
0


In [21]:
print(fit_120k.best.min())
print(fit_120k.best.max())

0.021055303476944898
0.08942185507880501


In [22]:
## Mutation strength
mut_sig = 5e-4
# Restarting seed
rnd.seed(42)

In [23]:
%%time
logbook.clear()

if __name__ == "__main__":
    for rep_n in range(rep_end):
        rep_seed = rnd.randint(0, 999)
        rnd.seed(rep_seed)
        # We initialize the population and evaluate the individuals' fitnesses
        pop = toolbox.population(n=pop_size)
        fitnesses = toolbox.map(toolbox.evaluate, pop)
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit
        # We start the logbook
        record = stat.compile(pop)
        births = len(pop)
        logbook.record(
            rep=rep_n + 1, 
            seed=rep_seed, 
            births=births, 
            **record)

        while births < births_end:
            # Select Parents and clone them as base for offsprings
            parents = toolbox.par_select(pop, k=par_size, tournsize=k_par)
            offspring = [toolbox.clone(ind) for ind in parents]
            births = births + len(offspring)

            # Aplly crossover
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                toolbox.mate(child1, child2, indpb=cx_pb)

            # Apply mutation
            for mutant in offspring:
                toolbox.mutate(mutant, sigma=mut_sig, indpb=mut_pb)
                del mutant.fitness.values

            fitnesses = toolbox.map(toolbox.evaluate, offspring)
            for ind, fit in zip(offspring, fitnesses):
                ind.fitness.values = fit

            pop = toolbox.sur_select((pop + offspring), k=pop_size, tournsize=k_sur)

            record = stat.compile(pop)
            logbook.record(
                rep=rep_n + 1, 
                seed=rep_seed, 
                births=births + 1, 
                **record)

Wall time: 7min 20s


In [24]:
fitness_res = pd.DataFrame.from_dict(logbook)
fitness_res = fitness_res[
    ["rep", "seed", "births", "avg", "std", "med", "worst", "best"]
]
fitness_res.head()

,rep,seed,births,avg,std,med,worst,best
0,1,654,160,2.303040,0.837745,2.273725,4.301838,0.380553
1,1,654,481,0.986895,0.286546,1.052427,1.746848,0.380553
2,1,654,801,0.481102,0.127213,0.423306,0.865288,0.279257
3,1,654,1121,0.362194,0.035483,0.379357,0.382517,0.276279
4,1,654,1441,0.286815,0.026557,0.279100,0.379842,0.271726


In [25]:
query = fitness_res["births"] >= births_end
fit_120k = fitness_res[query]
query = fitness_res["births"] == 60001
fit_60k = fitness_res[query]
query = fitness_res["births"] == 30241
fit_30k = fitness_res[query]

query_exact = fit_30k["best"] < 1e-6
fit_30k_exact = fit_30k[query_exact]
query_exact = fit_60k["best"] < 1e-6
fit_60k_exact = fit_60k[query_exact]
query_exact = fit_120k["best"] < 1e-6
fit_120k_exact = fit_120k[query_exact]

In [26]:
print(len(fit_30k))
print(len(fit_60k))
print(len(fit_120k))

print(len(fit_30k_exact))
print(len(fit_60k_exact))
print(len(fit_120k_exact))

40
40
40
0
0
0


In [27]:
print(fit_120k.best.min())
print(fit_120k.best.max())

0.0054711849483679575
0.23567870461192253


From OFAT

In [28]:
# Algorithm parameters
# Number of replicates, and generations per experiment
rep_end = 40
births_end = 120e3

# Genes
gen_size = 2
# Population size
pop_size = 160
# Progeny and parents size
b_ratio = 3
par_size = b_ratio * pop_size

# Progeny parameters
## Crossover probability per gene
cx_pb = 0.25
## Mutation probability per gene
mut_pb = 0.5
## Mutation strength
mut_sig = 1.25

# Selection by tournament
# Tournament size parent selection
k_par = 4
# Tournament size survivor selection
k_sur = 6

In [29]:
# Restarting seed
rnd.seed(42)

In [30]:
%%time
logbook.clear()

if __name__ == "__main__":
    for rep_n in range(rep_end):
        rep_seed = rnd.randint(0, 999)
        rnd.seed(rep_seed)
        # We initialize the population and evaluate the individuals' fitnesses
        pop = toolbox.population(n=pop_size)
        fitnesses = toolbox.map(toolbox.evaluate, pop)
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit
        # We start the logbook
        record = stat.compile(pop)
        births = len(pop)
        logbook.record(
            rep=rep_n + 1, 
            seed=rep_seed, 
            births=births, 
            **record)

        while births < births_end:
            # Select Parents and clone them as base for offsprings
            parents = toolbox.par_select(pop, k=par_size, tournsize=k_par)
            offspring = [toolbox.clone(ind) for ind in parents]
            births = births + len(offspring)

            # Aplly crossover
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                toolbox.mate(child1, child2, indpb=cx_pb)

            # Apply mutation
            for mutant in offspring:
                toolbox.mutate(mutant, sigma=mut_sig, indpb=mut_pb)
                del mutant.fitness.values

            fitnesses = toolbox.map(toolbox.evaluate, offspring)
            for ind, fit in zip(offspring, fitnesses):
                ind.fitness.values = fit

            pop = toolbox.sur_select((pop + offspring), k=pop_size, tournsize=k_sur)

            record = stat.compile(pop)
            logbook.record(
                rep=rep_n + 1, 
                seed=rep_seed, 
                births=births + 1, 
                **record)

Wall time: 6min 46s


In [31]:
fitness_res = pd.DataFrame.from_dict(logbook)
fitness_res = fitness_res[
    ["rep", "seed", "births", "avg", "std", "med", "worst", "best"]
]
fitness_res.head()

,rep,seed,births,avg,std,med,worst,best
0,1,654,160,2.303040,0.837745,2.273725,4.301838,0.380553
1,1,654,641,1.030528,0.256305,1.087489,1.716696,0.423220
2,1,654,1121,0.646550,0.153157,0.651217,1.000158,0.360033
3,1,654,1601,0.472623,0.103205,0.447761,0.963772,0.277700
4,1,654,2081,0.400260,0.077666,0.376788,0.691305,0.254370


In [32]:
query = fitness_res["births"] >= births_end
fit_120k = fitness_res[query]
query = fitness_res["births"] == 60001
fit_60k = fitness_res[query]
query = fitness_res["births"] == 30241
fit_30k = fitness_res[query]

query_exact = fit_30k["best"] < 1e-6
fit_30k_exact = fit_30k[query_exact]
query_exact = fit_60k["best"] < 1e-6
fit_60k_exact = fit_60k[query_exact]
query_exact = fit_120k["best"] < 1e-6
fit_120k_exact = fit_120k[query_exact]

In [33]:
print(len(fit_30k))
print(len(fit_60k))
print(len(fit_120k))

print(len(fit_30k_exact))
print(len(fit_60k_exact))
print(len(fit_120k_exact))

0
0
40
0
0
0


In [34]:
print(fit_120k.best.min())
print(fit_120k.best.max())

0.02036037389988221
0.10823705873188094


In [41]:
## Mutation strength
mut_sig = 5e-4
# Restarting seed
rnd.seed(42)

In [42]:
%%time
logbook.clear()

if __name__ == "__main__":
    for rep_n in range(rep_end):
        rep_seed = rnd.randint(0, 999)
        rnd.seed(rep_seed)
        # We initialize the population and evaluate the individuals' fitnesses
        pop = toolbox.population(n=pop_size)
        fitnesses = toolbox.map(toolbox.evaluate, pop)
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit
        # We start the logbook
        record = stat.compile(pop)
        births = len(pop)
        logbook.record(
            rep=rep_n + 1, 
            seed=rep_seed, 
            births=births, 
            **record)

        while births < births_end:
            # Select Parents and clone them as base for offsprings
            parents = toolbox.par_select(pop, k=par_size, tournsize=k_par)
            offspring = [toolbox.clone(ind) for ind in parents]
            births = births + len(offspring)

            # Aplly crossover
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                toolbox.mate(child1, child2, indpb=cx_pb)

            # Apply mutation
            for mutant in offspring:
                toolbox.mutate(mutant, sigma=mut_sig, indpb=mut_pb)
                del mutant.fitness.values

            fitnesses = toolbox.map(toolbox.evaluate, offspring)
            for ind, fit in zip(offspring, fitnesses):
                ind.fitness.values = fit

            pop = toolbox.sur_select((pop + offspring), k=pop_size, tournsize=k_sur)

            record = stat.compile(pop)
            logbook.record(
                rep=rep_n + 1, 
                seed=rep_seed, 
                births=births + 1, 
                **record)

Wall time: 7min 1s


In [43]:
fitness_res = pd.DataFrame.from_dict(logbook)
fitness_res = fitness_res[
    ["rep", "seed", "births", "avg", "std", "med", "worst", "best"]
]
fitness_res.head()

,rep,seed,births,avg,std,med,worst,best
0,1,654,160,2.303040,0.837745,2.273725,4.301838,0.380553
1,1,654,641,0.955305,0.307456,0.988815,1.794843,0.367720
2,1,654,1121,0.431305,0.115766,0.383236,0.988697,0.366991
3,1,654,1601,0.374904,0.007898,0.378235,0.381269,0.345158
4,1,654,2081,0.356256,0.013497,0.360348,0.379415,0.307810


In [44]:
query = fitness_res["births"] >= births_end
fit_120k = fitness_res[query]
query = fitness_res["births"] == 60001
fit_60k = fitness_res[query]
query = fitness_res["births"] == 30241
fit_30k = fitness_res[query]

query_exact = fit_30k["best"] < 1e-6
fit_30k_exact = fit_30k[query_exact]
query_exact = fit_60k["best"] < 1e-6
fit_60k_exact = fit_60k[query_exact]
query_exact = fit_120k["best"] < 1e-6
fit_120k_exact = fit_120k[query_exact]

In [45]:
print(len(fit_30k))
print(len(fit_60k))
print(len(fit_120k))

print(len(fit_30k_exact))
print(len(fit_60k_exact))
print(len(fit_120k_exact))

0
0
40
0
0
0


In [46]:
print(fit_120k.best.min())
print(fit_120k.best.max())

0.006297994041366439
0.28014480469145675
